In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

/var/folders/q7/gqrkvw7x1w9fqtxv3bsjbxwm0000gn/T/ipykernel_34061/1519389577.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
salary = pd.read_csv("survey_results_public.csv")
salary = salary[["Employment","RemoteWork" ,"EdLevel", "YearsCodePro", "DevType", "OrgSize", "Country","LanguageHaveWorkedWith", "DatabaseHaveWorkedWith", "PlatformHaveWorkedWith","ConvertedCompYearly"]]
print(len(salary))
print(salary.dtypes)

73268
Employment                 object
RemoteWork                 object
EdLevel                    object
YearsCodePro               object
DevType                    object
OrgSize                    object
Country                    object
LanguageHaveWorkedWith     object
DatabaseHaveWorkedWith     object
PlatformHaveWorkedWith     object
ConvertedCompYearly       float64
dtype: object


In [3]:
# cleaning operation 2: drop all rows with missing data ("NA")
salary = salary[pd.notna(salary["Employment"])]
salary = salary[pd.notna(salary["RemoteWork"])]
salary = salary[pd.notna(salary["EdLevel"])]
salary = salary[pd.notna(salary["YearsCodePro"])]
salary = salary[pd.notna(salary["DevType"])]
salary = salary[pd.notna(salary["OrgSize"])]
salary = salary[pd.notna(salary["Country"])]
salary = salary[pd.notna(salary["LanguageHaveWorkedWith"])]
salary = salary[pd.notna(salary["DatabaseHaveWorkedWith"])]
salary = salary[pd.notna(salary["PlatformHaveWorkedWith"])]
salary = salary[pd.notna(salary["ConvertedCompYearly"])]
print(len(salary))

26687


In [4]:
# cleaning operation 3: focus on data with full-time employment
salary = salary[salary["Employment"] == "Employed, full-time"]
print(len(salary))

21754


In [5]:
# cleaning operation 4: focus on data in USA
salary = salary[salary["Country"] == "United States of America"]
print(len(salary))

5633


In [6]:
# cleaning operation 5: focus on data for developers
salary = salary[salary["DevType"].str.contains("Developer")]
print(len(salary))

4901


In [7]:
# cleaning operation 6: drop invalid data
salary = salary[salary["ConvertedCompYearly"] > 50000]
print(len(salary))

4827


In [8]:
# cleaning operation 7: make "YearsCodePro" contain only numeric values
salary = salary.replace("Less than 1 year",0)
print(len(salary))

4827
